<a href="https://colab.research.google.com/github/min-hee-Lee/python_workdemo/blob/master/p405_pandas_%EC%98%81%ED%99%94%EC%B6%94%EC%B2%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###영화 추천 시스템

In [ ]:
#구글 코랩 한글
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 21 not upgraded.
Need to get 9,599 kB of archives.
After this operation, 29.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 fonts-nanum all 20180306-3 [9,599 kB]
Fetched 9,599 kB in 2s (5,466 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/python/data

Mounted at /content/drive
/content/drive/MyDrive/python/data


In [ ]:
!pip install folium
!pip install missingno

In [ ]:
#반드시 있어야되는것
import numpy as np
import pandas as pd
import matplotlib as mp
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#설치 후 import
import missingno as msno
import folium

### README

In [ ]:
with open('./m1/README') as fp:
  readme = fp.read()

In [ ]:
print(readme)

SUMMARY

These files contain 1,000,209 anonymous ratings of approximately 3,900 movies 
made by 6,040 MovieLens users who joined MovieLens in 2000.

USAGE LICENSE

Neither the University of Minnesota nor any of the researchers
involved can guarantee the correctness of the data, its suitability
for any particular purpose, or the validity of results based on the
use of the data set.  The data set may be used for any research
purposes under the following conditions:

     * The user may not state or imply any endorsement from the
       University of Minnesota or the GroupLens Research Group.

     * The user must acknowledge the use of the data set in
       publications resulting from the use of the data set, and must
       send us an electronic or paper copy of those publications.

     * The user may not redistribute the data without separate
       permission.

     * The user may not use this information for any commercial or
       revenue-bearing purposes without first obtaining 

### 데이터 가져오기

In [ ]:
#sep or delimiter : 필드를 구분하기 위해서 사용
# default : sep=','     delimiter=None
movies = pd.read_csv('./m1/movies.dat', delimiter='::', encoding='ISO-8859-1', engine='python', header=None)

In [ ]:
print(movies.shape)

(3883, 3)


In [ ]:
#MovieID::Title::Genres
movies.head()

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
movies.tail()

,0,1,2
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama
3882,3952,"Contender, The (2000)",Drama|Thriller


In [ ]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3883 entries, 0 to 3882
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       3883 non-null   int64 
 1   1       3883 non-null   object
 2   2       3883 non-null   object
dtypes: int64(1), object(2)
memory usage: 91.1+ KB


In [ ]:
movies.set_index(0)

,1,2
0,,
1,Toy Story (1995),Animation|Children's|Comedy
2,Jumanji (1995),Adventure|Children's|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama
5,Father of the Bride Part II (1995),Comedy
...,...,...
3948,Meet the Parents (2000),Comedy
3949,Requiem for a Dream (2000),Drama
3950,Tigerland (2000),Drama


In [ ]:
movies.head()

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
users = pd.read_csv('./m1/users.dat', sep="::", engine='python', header=None)

In [ ]:
#UserID::Gender::Age::Occupation::Zip-code
users.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [ ]:
users.shape

(6040, 5)

In [ ]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6040 entries, 0 to 6039
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       6040 non-null   int64 
 1   1       6040 non-null   object
 2   2       6040 non-null   int64 
 3   3       6040 non-null   int64 
 4   4       6040 non-null   object
dtypes: int64(3), object(2)
memory usage: 236.1+ KB


In [ ]:
ratings = pd.read_csv('./m1/ratings.dat', sep="::", engine='python', header=None)

In [ ]:
# UserID::MovieID::Rating::Timestamp
ratings.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [ ]:
ratings.shape

(1000209, 4)

In [ ]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000209 entries, 0 to 1000208
Data columns (total 4 columns):
 #   Column  Non-Null Count    Dtype
---  ------  --------------    -----
 0   0       1000209 non-null  int64
 1   1       1000209 non-null  int64
 2   2       1000209 non-null  int64
 3   3       1000209 non-null  int64
dtypes: int64(4)
memory usage: 30.5 MB


### 데이터 전처리(preprocessing)

In [ ]:
print(dir(movies))

['T', '_AXIS_LEN', '_AXIS_ORDERS', '_AXIS_REVERSED', '_AXIS_TO_AXIS_NUMBER', '_HANDLED_TYPES', '__abs__', '__add__', '__and__', '__annotations__', '__array__', '__array_priority__', '__array_ufunc__', '__array_wrap__', '__bool__', '__class__', '__contains__', '__copy__', '__deepcopy__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__divmod__', '__doc__', '__eq__', '__finalize__', '__floordiv__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__iadd__', '__iand__', '__ifloordiv__', '__imod__', '__imul__', '__init__', '__init_subclass__', '__invert__', '__ior__', '__ipow__', '__isub__', '__iter__', '__itruediv__', '__ixor__', '__le__', '__len__', '__lt__', '__matmul__', '__mod__', '__module__', '__mul__', '__ne__', '__neg__', '__new__', '__nonzero__', '__or__', '__pos__', '__pow__', '__radd__', '__rand__', '__rdivmod__', '__reduce__', '__reduce_ex__', '__repr__', '__rfloordiv__', '__rmatmul__', '__rmod__', '__rmul

README를 읽고 column이름을 변경한다

In [ ]:
movies.columns  #MovieID::Title::Genres

Int64Index([0, 1, 2], dtype='int64')

In [ ]:
#버전에 따라서는 컬럼명이 숫자일 때 문자열로 써야하는 경우도 있다.
movies.rename(columns={0:'MovieID', 1:'Title', 2:'Genres'}, inplace=True)

In [ ]:
movies.head()

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
# UserID::Gender::Age::Occupation::Zip-code

#column 이름을 바꿀 때는 column 또는 axis=1을 사용한다.
#users.rename(column={0:'UserID', 1:'Gender', 2:'Age', 3:'Occupation', 4:'Zip-code'}, inplace=True)
#users.rename(mapper={0:'UserID', 1:'Gender', 2:'Age', 3:'Occupation', 4:'Zip-code'}, axis=1, inplace=True)

#데이터를 읽어올 때 column 이름을 지정할 수 있다.
users = pd.read_csv('./m1/users.dat', sep="::", engine='python', header=None,
                    names=['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code'])

In [ ]:
users.head()

,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [ ]:
# UserID::MovieID::Rating::Timestamp


#ratings.rename(columns={0:'USERID', 1:'MovieID', 2:'Rating', 3:'Timestamp'}, inplace=True)
#ratings.rename(mapper={0:'USERID', 1:'MovieID', 2:'Rating', 3:'Timestamp'}, axis=1, inplace=True)


ratings = pd.read_csv('./m1/ratings.dat', 
                     sep='::', 
                    engine='python', 
                    header=None,
          names=['UserID', 'MovieID', 'Rating', 'Timestamp'])

In [ ]:
ratings.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


###데이터 구조 변경

In [ ]:
movies.columns

Index(['MovieID', 'Title', 'Genres'], dtype='object')

In [ ]:
users.columns

Index(['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code'], dtype='object')

In [ ]:
ratings.columns

Index(['UserID', 'MovieID', 'Rating', 'Timestamp'], dtype='object')

### merge와 join비교

In [ ]:
df = pd.DataFrame({'key':['K0','K1','K2','K3','K4','K5'],
                   'A':['A0','A1','A2','A3','A4','A5']})
df

,key,A
0,K0,A0
1,K1,A1
2,K2,A2
3,K3,A3
4,K4,A4
5,K5,A5


In [ ]:
other = pd.DataFrame({'key':['K0','K1','K2'],
                   'B':['B0','B1','B2']})
other

,key,B
0,K0,B0
1,K1,B1
2,K2,B2


In [ ]:
# join()은 how속성의 값이 left이므로 left outer join을 실행한다.
# left outer join
df.join(other.set_index('key'), on='key')

,key,A,B
0,K0,A0,B0
1,K1,A1,B1
2,K2,A2,B2
3,K3,A3,NaN
4,K4,A4,NaN
5,K5,A5,NaN


In [ ]:
# how = left
# Left Outer Join
other.join(df.set_index('key'), on='key')

,key,B,A
0,K0,B0,A0
1,K1,B1,A1
2,K2,B2,A2


In [ ]:
# how속성의 기본값이 inner이다.
# inner join
df.merge(other)  

,key,A,B
0,K0,A0,B0
1,K1,A1,B1
2,K2,A2,B2


merge는 이름이 같은 column을 기준으로 데이터를 합친다.

In [ ]:
data = ratings.merge(users).merge(movies)

In [ ]:
data.head()

,UserID,MovieID,Rating,Timestamp,Gender,Age,Occupation,Zip-code,Title,Genres
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,M,56,16,70072,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,M,25,12,32793,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,M,25,7,22903,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,M,50,1,95350,One Flew Over the Cuckoo's Nest (1975),Drama


In [ ]:
data.columns

Index(['UserID', 'MovieID', 'Rating', 'Timestamp', 'Gender', 'Age',
       'Occupation', 'Zip-code', 'Title', 'Genres'],
      dtype='object')

In [ ]:
# fancy indexing 해서 필요한 컬럼만 가져온다.
recommendation_data = data[['UserID','MovieID', 'Rating']].copy()

In [ ]:
recommendation_data.head()

,UserID,MovieID,Rating
0,1,1193,5
1,2,1193,5
2,12,1193,4
3,15,1193,4
4,17,1193,5


In [ ]:
recommendation_data.shape

(1000209, 3)

join
- join은 index를 기준으로 데이터를 합쳐준다.
- merge와 비슷하지만 중복되는 컬럼이 있으면 이름 구분자를 정해줘야 한다.(rsuffix, lsuffix)

In [ ]:
movies.columns

Index(['MovieID', 'Title', 'Genres'], dtype='object')

In [ ]:
users.columns

Index(['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code'], dtype='object')

In [ ]:
ratings.columns

Index(['UserID', 'MovieID', 'Rating', 'Timestamp'], dtype='object')

In [ ]:
ratings.head(1)

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760


In [ ]:
users.head(1)

,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067


In [ ]:
#on = 'UserID'는 ratings에서 UserID 컬럼을 기준으로 삼는 옵션이다.
# other에서는 UserID를 인덱스로 사용한다.
# 즉, ratings와 users 모두 UserID를 기준으로 join한다
ratings.join(other = users.set_index('UserID'), on='UserID')

,UserID,MovieID,Rating,Timestamp,Gender,Age,Occupation,Zip-code
0,1,1193,5,978300760,F,1,10,48067
1,1,661,3,978302109,F,1,10,48067
2,1,914,3,978301968,F,1,10,48067
3,1,3408,4,978300275,F,1,10,48067
4,1,2355,5,978824291,F,1,10,48067
...,...,...,...,...,...,...,...,...
1000204,6040,1091,1,956716541,M,25,6,11106
1000205,6040,1094,5,956704887,M,25,6,11106
1000206,6040,562,5,956704746,M,25,6,11106
1000207,6040,1096,4,956715648,M,25,6,11106
